In [1]:
import os
import matplotlib.pyplot as plt
import re
import numpy as np
from scipy import stats
#import optshrink as opt # package we create
import numpy as np
# import scipy.io as sio
import h5py
import matplotlib.pyplot as plt 
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import seaborn as sns
from shapely.geometry import Point
import geopandas as gp
from geodatasets import get_path
from shapely.geometry import Polygon

In [2]:
region = 'region_1'

file_path = '/group/moniergrp/TEM_Large_Ensemble/input_files/large_ensemble/'



In [3]:

pre_atm_chem = pd.DataFrame(os.listdir(file_path+region+'/atm_chem/pre_data') , columns = ['file'])
pre_clim = pd.DataFrame(os.listdir(file_path+region+'/climate/pre_data') , columns = ['file'])
pre_lulcc = pd.DataFrame(os.listdir(file_path+region+'/lulcc/pre_data'), columns = ['file'])


hist_atm_chem = pd.DataFrame(os.listdir(file_path+region+'/atm_chem/historical') , columns = ['file'])
hist_clim = pd.DataFrame(os.listdir(file_path+region+'/climate/historical') , columns = ['file'])
hist_lulcc = pd.DataFrame(os.listdir(file_path+region+'/lulcc/historical'), columns = ['file'])


future_atm_chem = pd.DataFrame(os.listdir(file_path+region+'/atm_chem/future') , columns = ['file'])
future_clim = pd.DataFrame(os.listdir(file_path+region+'/climate/future') , columns = ['file'])
future_lulcc = pd.DataFrame(os.listdir(file_path+region+'/lulcc/future'), columns = ['file'])



In [4]:
pre_atm_chem[['type', 'scen']] = pre_atm_chem['file'].str.split('_', expand=True)
pre_atm_chem = pre_atm_chem.loc[~pre_atm_chem['file'].str.contains('RCP')]
pre_atm_chem.loc[pre_atm_chem['type']=='Nfer', 'type'] = pre_atm_chem['type'] + pre_atm_chem['scen']

hist_atm_chem[['type', 'scen']] = hist_atm_chem['file'].str.split('_', expand=True)
hist_atm_chem = hist_atm_chem.loc[~hist_atm_chem['file'].str.contains('RCP')]
hist_atm_chem.loc[hist_atm_chem['type']=='Nfer', 'type'] = hist_atm_chem['type'] + hist_atm_chem['scen']

future_atm_chem[['type', 'scen']] = future_atm_chem['file'].str.split('_', expand=True)
future_atm_chem = future_atm_chem.loc[~future_atm_chem['file'].str.contains('RCP')]
future_atm_chem.loc[future_atm_chem['type']=='Nfer', 'type'] = future_atm_chem['type'] + future_atm_chem['scen']

In [19]:
read_chem = pre_atm_chem.merge(
    hist_atm_chem, on = 'type', suffixes = ('_pre','_hist')).merge(
        future_atm_chem
                            )

read_chem['group'] = 'atm_chem'

In [25]:
pre_clim[['type', 'scen', 'var']] = pre_clim['file'].str.split('_', expand=True)
pre_clim['type'] = pre_clim['type'] + pre_clim['var'] 

hist_clim[['type', 'scen', 'var']] = hist_clim['file'].str.split('_', expand=True)
hist_clim['type'] = hist_clim['type'] + hist_clim['var'] 

future_clim[['type', 'scen', 'var']] = future_clim['file'].str.split('_', expand=True)
future_clim['type'] = future_clim['type'] + future_clim['var'] 

read_clim = pre_clim.merge(
    hist_clim, on = 'type', suffixes = ('_pre','_hist')).merge(
        future_clim
                            )

read_clim['group'] = 'climate'
read_clim

,file_pre,type,scen_pre,var_pre,file_hist,scen_hist,var_hist,file,scen,var,group
0,MPIxxESM1xx2xxLR_ssp245_prec.csv,MPIxxESM1xx2xxLRprec.csv,ssp245,prec.csv,MPIxxESM1xx2xxLR_ssp245_prec.csv,ssp245,prec.csv,MPIxxESM1xx2xxLR_ssp370_prec.csv,ssp370,prec.csv,climate
1,MPIxxESM1xx2xxLR_ssp245_prec.csv,MPIxxESM1xx2xxLRprec.csv,ssp245,prec.csv,MPIxxESM1xx2xxLR_ssp245_prec.csv,ssp245,prec.csv,MPIxxESM1xx2xxLR_ssp126_prec.csv,ssp126,prec.csv,climate
2,MPIxxESM1xx2xxLR_ssp245_prec.csv,MPIxxESM1xx2xxLRprec.csv,ssp245,prec.csv,MPIxxESM1xx2xxLR_ssp245_prec.csv,ssp245,prec.csv,MPIxxESM1xx2xxLR_ssp585_prec.csv,ssp585,prec.csv,climate
3,MPIxxESM1xx2xxLR_ssp245_prec.csv,MPIxxESM1xx2xxLRprec.csv,ssp245,prec.csv,MPIxxESM1xx2xxLR_ssp245_prec.csv,ssp245,prec.csv,MPIxxESM1xx2xxLR_ssp245_prec.csv,ssp245,prec.csv,climate
4,CanESM5_ssp245_wind.csv,CanESM5wind.csv,ssp245,wind.csv,CanESM5_ssp245_wind.csv,ssp245,wind.csv,CanESM5_ssp245_wind.csv,ssp245,wind.csv,climate
...,...,...,...,...,...,...,...,...,...,...,...
199,MRIxxESM2xx0_ssp245_prec.csv,MRIxxESM2xx0prec.csv,ssp245,prec.csv,MRIxxESM2xx0_ssp245_prec.csv,ssp245,prec.csv,MRIxxESM2xx0_ssp460_prec.csv,ssp460,prec.csv,climate
200,MPIxxESM1xx2xxHR_ssp245_trange.csv,MPIxxESM1xx2xxHRtrange.csv,ssp245,trange.csv,MPIxxESM1xx2xxHR_ssp245_trange.csv,ssp245,trange.csv,MPIxxESM1xx2xxHR_ssp126_trange.csv,ssp126,trange.csv,climate
201,MPIxxESM1xx2xxHR_ssp245_trange.csv,MPIxxESM1xx2xxHRtrange.csv,ssp245,trange.csv,MPIxxESM1xx2xxHR_ssp245_trange.csv,ssp245,trange.csv,MPIxxESM1xx2xxHR_ssp585_trange.csv,ssp585,trange.csv,climate
202,MPIxxESM1xx2xxHR_ssp245_trange.csv,MPIxxESM1xx2xxHRtrange.csv,ssp245,trange.csv,MPIxxESM1xx2xxHR_ssp245_trange.csv,ssp245,trange.csv,MPIxxESM1xx2xxHR_ssp370_trange.csv,ssp370,trange.csv,climate


In [10]:
types = read_chem.type.unique()

In [16]:
ty = types[0]
i=0

In [22]:
sub_in = read_chem.loc[read_chem['type']==ty].reset_index()
sub_in

###read in first row for pre data and historical, which are repeated in future scenarios
pre_data = pd.read_csv(file_path+region+'/'+sub_in.loc[0,'group']+'/pre_data/'+sub_in.loc[0,'file_pre'])
historical = pd.read_csv(file_path+region+'/'+sub_in.loc[0,'group']+'/historical/'+sub_in.loc[0,'file_hist'])

for i in range(len(sub_in)):
    future = pd.read_csv(file_path+region+'/'+sub_in.loc[0,'group']+'/historical/'+sub_in.loc[0,'file_hist'])

In [32]:
future_atm_chem.loc[future_atm_chem['file'].str.contains('rcp')]

,file,type,scen
0,AOT40_rcp60.csv,AOT40,rcp60.csv
2,Ndep_rcp45.csv,Ndep,rcp45.csv
4,Ndep_rcp85.csv,Ndep,rcp85.csv
7,Ndep_rcp26.csv,Ndep,rcp26.csv
8,AOT40_rcp85.csv,AOT40,rcp85.csv
9,AOT40_rcp26.csv,AOT40,rcp26.csv
10,Ndep_rcp60.csv,Ndep,rcp60.csv
12,AOT40_rcp45.csv,AOT40,rcp45.csv


In [33]:
hist_atm_chem.loc[hist_atm_chem['file'].str.contains('rcp')]

,file,type,scen
0,AOT40_rcp26.csv,AOT40,rcp26.csv
3,Ndep_rcp26.csv,Ndep,rcp26.csv


In [18]:
pre_atm_chem['file'].str.split('_')

0       [Nfer, pas.csv]
1     [Ndep, rcp26.csv]
2      [Nfer, crop.csv]
3    [AOT40, RCP26.csv]
4    [AOT40, rcp26.csv]
Name: file, dtype: object

In [14]:
hist_atm_chem

,file
0,AOT40_rcp26.csv
1,Nfer_pas.csv
2,AOT40_RCP26.csv
3,Ndep_rcp26.csv
4,Nfer_crop.csv


In [15]:
future_atm_chem

,file
0,AOT40_rcp60.csv
1,AOT40_RCP45.csv
2,Ndep_rcp45.csv
3,Nfer_crop.csv
4,Ndep_rcp85.csv
5,AOT40_RCP85.csv
6,AOT40_RCP26.csv
7,Ndep_rcp26.csv
8,AOT40_rcp85.csv
9,AOT40_rcp26.csv
